## Librerias

In [2]:
import time
import re
import io
import csv
import json
import traceback
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
from azure.storage.blob import BlobServiceClient
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain.docstore.document import Document
import os
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

load_dotenv()

True

## Inicializamos la página

In [4]:
resultados_texto = ""
fecha_hoy = datetime.now().strftime('%Y-%m-%d')
nombre_archivo_txt = f'mercadonafinal_{fecha_hoy}.txt'
contenido_csv = []


AZURE_API_KEY = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY =  os.getenv("OPENAI_API_KEY")
usuario_email_mercadona = os.getenv('USUARIO_MERCADONA')
contrasena_mercadona = os.getenv('CONTRASENA_MERCADONA')

def get_blob_client(nombre_archivo):
    connect_str = AZURE_API_KEY
    container_name = "scrapingmercadona"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=nombre_archivo)
    return blob_client

In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

service = webdriver.chrome.service.Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service) 
driver.get("https://www.mercadona.es/")


In [4]:
service = Service(executable_path='chromedriver.exe')
options = webdriver.ChromeOptions() 
options.add_argument("--disable-blink-features=AutomationControlled") 
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False) 
driver = webdriver.Chrome(options=options,service=service) 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
driver.get("https://www.mercadona.es/")

elemento = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[2]/header/div/div/form/div/input")))

cp = driver.find_element(By.XPATH,"/html/body/div[2]/header/div/div/form/div/input").send_keys('29002')
cp = driver.find_element(By.XPATH,"/html/body/div[2]/header/div/div/form/input").click()

time.sleep(2)
usuario_input = driver.find_element(By.XPATH,"/html/body/div[5]/div/div/div[2]/form/div[2]/div/div[1]/span/input")
contrasena_input = driver.find_element(By.XPATH,"/html/body/div[5]/div/div/div[2]/form/div[2]/div/div[2]/span/input")



usuario_input.send_keys(usuario_email_mercadona)  # Completar el campo de usuario
contrasena_input.send_keys(contrasena_mercadona)  # Completar el campo de contraseña


# Enviar el formulario (si es necesario)
contrasena_input.send_keys(Keys.RETURN)  # Presionar Enter para enviar el formulario




## Definimos la función que extrae información

In [5]:
def Jacques_Mayol():
    patron_fila = r"InsertaLinea\((.*?)\);"
    coincidencias = re.findall(patron_fila, driver.page_source)
    resultados = ""
    
    for fila in coincidencias:
        data_file = io.StringIO(fila)

        # Usamos el módulo csv de Python para leer los datos
        reader = csv.reader(data_file, delimiter=',')

        # Iteramos sobre las filas (en este caso solo hay una)
        for row in reader:
            # Utilizamos expresiones regulares para encontrar el precio por kilo
            price_per_kilo_match = re.search(r'\d+,\d+', row[12])
            if price_per_kilo_match:
                price_per_kilo = float(price_per_kilo_match.group().replace(',', '.'))  # Reemplazamos ',' por '.' y convertimos a float
            else:
                price_per_kilo = None

            selected_data_text = (f"Id:{row[0]} %% El producto del supermercado MERCADONA :{row[3]} %% tiene un precio de €:{row[4]}")
            resultados += selected_data_text + " // "

    return resultados

In [6]:
def print_all_frames(driver, parent_frame=None, indent=0):
    # Obtiene todos los frames presentes en la página dentro del frame padre dado
    frames = driver.find_elements(By.TAG_NAME,'frame') + driver.find_elements(By.TAG_NAME,'iframe')
    return frames
    # Imprime los nombres e IDs de los frames

In [7]:
def busca_paginas():
    patron_paginas = r'Página (\d+) de (\d+)'
    codigo_fuente = driver.page_source
    coincidencias = re.search(patron_paginas, codigo_fuente)

    if coincidencias:
        pagina_actual = int(coincidencias.group(1))
        total_paginas = int(coincidencias.group(2))
        print("Página actual:", pagina_actual)
        print("Total de páginas:", total_paginas)
        if pagina_actual != total_paginas:
            boton_siguiente = driver.find_element(By.XPATH, "//a[contains(text(), 'Página siguiente')]")
            boton_siguiente.click()
            return True
        else:
            return False
    else:
        print("No se encontró información sobre el número de páginas.")
        return False

In [8]:
def escribir_datos_en_archivo():
    global resultados_texto
    
    # Obtener la fecha de hoy en el formato YYYY-MM-DD
    fecha_hoy = datetime.now().strftime("%Y-%m-%d")

    # Abrir el archivo en modo append (agregar)
    with open(nombre_archivo_txt, 'a') as archivo:
        archivo.write(resultados_texto)
        resultados_texto = ""  # Limpiar después de escribir

def subir_archivo_al_blob(nombre_archivo_txt):
    blob_client = get_blob_client(nombre_archivo_txt)
    

    with open(nombre_archivo_txt, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

In [9]:
def procesar_campo(campo):
    try:
        valor = campo.split(":")[1].strip()
    except IndexError:
        valor = campo
    return valor
        
        

def cargar_csv_blob():
    global contenido_csv
    fecha_hoy = datetime.now().strftime("%Y-%m-%d")
    nombre_archivo_csv = f"{fecha_hoy}_productos.csv"
    blob_client = get_blob_client(nombre_archivo_csv)
    try:
        # Descargar el archivo CSV existente
        with open(nombre_archivo_csv, "wb") as archivo_local:
            blob_data = blob_client.download_blob()
            archivo_local.write(blob_data.readall())

        # Leer el contenido del archivo CSV existente
        with open(nombre_archivo_csv, mode='r', encoding='utf-8') as archivo_csv:
            lector_csv = csv.reader(archivo_csv)
            contenido_csv.extend(list(lector_csv))
    except Exception as e:
        print(f"Archivo CSV no existe o no se puede cargar: {e}")
        # Si el archivo no existe, crear una cabecera para el CSV
        contenido_csv.append(["Categoria", "Subcategoria", "Subsubcategoria", "Fecha", "ID", "Nombre", "Precio"])
        
def guardar_csv_blob():
    fecha_hoy = datetime.now().strftime("%Y-%m-%d")
    nombre_archivo_csv = f"{fecha_hoy}_productos.csv"
    
    blob_client = get_blob_client(nombre_archivo_csv)

    # Guardar el contenido en un archivo CSV
    with open(nombre_archivo_csv, mode='w', newline='', encoding='utf-8') as archivo_csv:
        escritor_csv = csv.writer(archivo_csv)
        escritor_csv.writerows(contenido_csv)

    # Subir el archivo CSV al blob storage

    with open(nombre_archivo_csv, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)


In [10]:
def procesar_paginas(categoria, subcategoria="", subsubcategoria=""):
    global resultados_texto,resultados_texto_temp
    while True:
        resultados = Jacques_Mayol()
        resultados_texto += resultados
        time.sleep(1)
        resultados_texto_temp = resultados_texto
        escribir_datos_en_archivo()
        procesar_resultados(resultados_texto_temp, categoria, subcategoria, subsubcategoria)
        if not busca_paginas():
            resultados_texto_temp = resultados_texto
            escribir_datos_en_archivo()
            procesar_resultados(resultados_texto_temp, categoria, subcategoria, subsubcategoria)
            break
            
def procesar_resultados(resultados_texto_temp, categoria, subcategoria="", subsubcategoria=""):
    fecha_hoy = datetime.now().strftime("%Y-%m-%d")
    
    # Procesar cada línea de resultados_texto_temp
    for linea in resultados_texto_temp.split(" // "):
        print(linea)
        campos = linea.split(" %% ")
        if len(campos) >= 3:
            # Extraer los campos relevantes
            id_producto = procesar_campo(campos[0])
            nombre = procesar_campo(campos[1])
            precio = procesar_campo(campos[2])

            # Verificar si el ID del producto ya existe en el archivo CSV
            ids_registrados = set([fila[4] for fila in contenido_csv[1:]])  # Ignorar la cabecera
            if id_producto not in ids_registrados:
                # Agregar el producto al contenido del CSV
                contenido_csv.append([categoria, subcategoria, subsubcategoria, fecha_hoy, id_producto, nombre, precio])

    
    

## Función que recorre las páginas del mercadona

In [11]:
try:
    driver.switch_to.default_content()
    driver.switch_to.frame('toc')
    driver.switch_to.frame('menu')
    categorias_tags = driver.find_elements(By.TAG_NAME, "a")
    categorias_lista = [enlace.text for enlace in categorias_tags]
    print(f"Las categorias iniciales son {categorias_lista}")
    
    cargar_csv_blob()

    for categoria in categorias_lista:
        print("Estoy empezando")
        print(f'Buceando en {categoria}')
        print(f"Voy a hacer click en {categoria}")
        driver.find_element(By.XPATH, f"//a[contains(text(),'{categoria}')]").click()
        time.sleep(1)
        subcategorias = [enlace.text for enlace in driver.find_elements(By.TAG_NAME, "a") if enlace.text not in categorias_lista]
        print(subcategorias)
        if not subcategorias:
            print(f"No existen las subcategorias, te guardo la categoria {categoria}")
            driver.switch_to.default_content()
            driver.switch_to.frame('mainFrame')
            procesar_paginas(categoria)
            driver.switch_to.default_content()
            driver.switch_to.frame('toc')
            driver.switch_to.frame('menu')
            time.sleep(1)
        else:
            print(f"Las subcategorias son {subcategorias}")
            for subcategoria in subcategorias:
                print(f"Las subcategoria es {subcategoria}")
                print(f"Voy a hacer click en {subcategoria}")
                try:
                    driver.find_element(By.XPATH, f"//a[text()='{subcategoria}']").click()
                except:
                    driver.find_element(By.XPATH, f"//a[normalize-space(text())='{subcategoria}']").click()
                subsubcategorias = [enlace.text for enlace in driver.find_elements(By.TAG_NAME, "a") if enlace.text not in categorias_lista + subcategorias]
                if not subsubcategorias:
                    print(f"No existen subsubcategorias, te guardo la subcategoria {subcategoria}")
                    driver.switch_to.default_content()
                    driver.switch_to.frame('mainFrame')
                    time.sleep(1)
                    procesar_paginas(categoria, subcategoria)
                    driver.switch_to.default_content()
                    driver.switch_to.frame('toc')
                    driver.switch_to.frame('menu')
                    time.sleep(1)
                else:
                    print(f"Las subsubcategorias son {subsubcategorias}")
                    for subsubcategoria in subsubcategorias:
                        print(f"Las primer subsubcategoria es {subsubcategoria}")
                        print(f"Voy a hacer click en {subsubcategoria}")
                        try:
                            driver.find_element(By.XPATH, f"//a[text()='{subsubcategoria}']").click()
                        except:
                            driver.find_element(By.XPATH, f"//a[normalize-space(text())='{subsubcategoria}']").click()
                        driver.switch_to.default_content()
                        driver.switch_to.frame('mainFrame')
                        procesar_paginas(categoria, subcategoria, subsubcategoria)
                        driver.switch_to.default_content()
                        driver.switch_to.frame('toc')
                        driver.switch_to.frame('menu')

except (Exception, KeyboardInterrupt) as e:
    print("Se produjo un error:", e)
    print(traceback.format_exc())
    # Podrías querer subir el archivo general al Blob Storage en caso de fallo
    guardar_csv_blob()
    subir_archivo_al_blob(nombre_archivo_txt)
finally:
    guardar_csv_blob()
    subir_archivo_al_blob(nombre_archivo_txt)
    #driver.quit()

Las categorias iniciales son ['BAJADA PVP', 'NOVEDADES', 'SUSHI', 'AGUA Y REFRESCOS', 'ALIMENTACION', 'APERITIVOS', 'BEBES', 'BODEGA', 'CARNES', 'CHARCUTERIA', 'COMPLEMENTOS DE HOGAR', 'CONGELADOS', 'DROGUERIA', 'FRUTAS Y VERDURAS', 'HORNO Y BOLLERIA', 'LACTEOS, BEBIDAS Y POSTRES VEGETALES', 'MASCOTAS', 'PERFUMERIA', 'PESCADERIA', 'ZUMOS']
Archivo CSV no existe o no se puede cargar: The specified blob does not exist.
RequestId:c443ef06-001e-001a-067f-ae15db000000
Time:2024-05-25T08:44:21.9264629Z
ErrorCode:BlobNotFound
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:c443ef06-001e-001a-067f-ae15db000000
Time:2024-05-25T08:44:21.9264629Z</Message></Error>
Estoy empezando
Buceando en BAJADA PVP
Voy a hacer click en BAJADA PVP
[]
No existen las subcategorias, te guardo la categoria BAJADA PVP
Id:4640 %% El producto del supermercado MERCADONA :ACEITE OLIVA SABOR INTENSO TAPON VERDE  ***BAJADA PVP PENINSULA 

Id:13406 %% El producto del supermercado MERCADONA :MAYONESA ***BAJADA PVP PENINSULA Y BALEARES***, HACENDADO, TARRO (BOCA ANCHA) 500 ml %% tiene un precio de €:1.2
Id:13407 %% El producto del supermercado MERCADONA :MAYONESA  ***BAJADA PVP***, HACENDADO, BOTE 460 ml %% tiene un precio de €:1.25
Id:35932 %% El producto del supermercado MERCADONA :MERMELADA ARANDANOS ***BAJADA PVP CANARIAS***, HACENDADO, TARRO 330 g %% tiene un precio de €:1.95
Id:86755 %% El producto del supermercado MERCADONA :MERMELADA FRESA ***BAJADA PVP CANARIAS***, HACENDADO, TARRO 400 g %% tiene un precio de €:1.55
Id:68243 %% El producto del supermercado MERCADONA :NATILLA CHOCOLATE DANET***BAJADA PVP PENINSULA Y BALEARES***, DANONE, PACK 4 x 120 g - 480 g %% tiene un precio de €:1.8
Id:86245 %% El producto del supermercado MERCADONA :NATILLA VAINILLA DANET ***BAJADA PVP PENINSULA Y BALEARES***, DANONE, PACK 4 x 120 g - 480 g %% tiene un precio de €:1.8
Id:6305 %% El producto del supermercado MERCADONA :PAJARITA

Id:52622 %% El producto del supermercado MERCADONA :ACEITUNA GORDAL PICADEDOS ALIÑADA SIN HUESO *****NOVEDAD*****, HACENDADO, TARRO 710 g ESCURRIDO 350 g %% tiene un precio de €:3.25
Id:86339 %% El producto del supermercado MERCADONA :BARRITAS CON GALLETA CAOCREM *****NOVEDAD*****, HACENDADO, PAQUETE 5 u - 105 g %% tiene un precio de €:1.5
Id:52451 %% El producto del supermercado MERCADONA :BOQUERONES ALIÑADOS *****NOVEDAD*****, HACENDADO, PAQUETE 80 g ESCURRIDO 60 g %% tiene un precio de €:1.8
Id:1564 %% El producto del supermercado MERCADONA :CERDO COSTILLA IBERICO ADOBADA FRESCO *****NOVEDAD*****, LA HACIENDA, PAQUETE 600 g APROX(PESO APROXIMADO DE LA UNIDAD 600 GR) %% tiene un precio de €:4.11
Id:60551 %% El producto del supermercado MERCADONA :CHOCOLATINAS SNACKS MINI MIX *****NOVEDAD*****, HACENDADO, PAQUETE 395 g %% tiene un precio de €:2.95
Id:60541 %% El producto del supermercado MERCADONA :CREMA BISCUIT *****NOVEDAD*****, HACENDADO, BOTE 400 g %% tiene un precio de €:2.6
Id:8

Id:28092 %% El producto del supermercado MERCADONA :AGUA MINERAL NATURAL, FONT NATURA, BOTELLA 1 l %% tiene un precio de €:0.4
Id:27691 %% El producto del supermercado MERCADONA :AGUA MINERAL NATURAL, FONT NATURA, PACK 6 x 500 ml - 3 l %% tiene un precio de €:1.32
Id:27692 %% El producto del supermercado MERCADONA :AGUA MINERAL NATURAL, FONT NATURA, BOTELLA 500 ml %% tiene un precio de €:0.22
Id:28089 %% El producto del supermercado MERCADONA :AGUA MINERAL NATURAL, FONT NATURA, PACK 6 x 1500 ml - 9 l %% tiene un precio de €:1.5
Id:28090 %% El producto del supermercado MERCADONA :AGUA MINERAL NATURAL, FONT NATURA, BOTELLA 1500 ml %% tiene un precio de €:0.25
Id:27350 %% El producto del supermercado MERCADONA :AGUA MINERAL NATURAL, FONTVELLA, PACK 6 x 1500 ml - 9 l %% tiene un precio de €:4.2
Id:27204 %% El producto del supermercado MERCADONA :AGUA MINERAL NATURAL, FONTVELLA, BOTELLA 1500 ml %% tiene un precio de €:0.73
Id:27208 %% El producto del supermercado MERCADONA :AGUA MINERAL NAT

Las subsubcategorias son ['COLA CLASICA', 'COLA ZERO', 'COLA SIN CAFEINA']
Las primer subsubcategoria es COLA CLASICA
Voy a hacer click en COLA CLASICA
Id:27321 %% El producto del supermercado MERCADONA :COLA NORMAL ***BAJADA PVP CANARIAS***, COCA-COLA, LATA PACK 12 x 330 ml - 3960 ml %% tiene un precio de €:10.44
Id:27348 %% El producto del supermercado MERCADONA :COLA NORMAL  ***BAJADA PVP***, COCA-COLA, PACK 2 x 2 l - 4 l %% tiene un precio de €:3.7
Id:28113 %% El producto del supermercado MERCADONA :COLA NORMAL ***PACK AHORRO***, COCA-COLA, BOTELLA PACK 4 x 500 ml - 2 l %% tiene un precio de €:5
Id:27342 %% El producto del supermercado MERCADONA :COLA NORMAL, COCA-COLA, BOTELLA 2 l %% tiene un precio de €:1.99
Id:28181 %% El producto del supermercado MERCADONA :COLA NORMAL, COCA-COLA, BOTELLA 1250 ml %% tiene un precio de €:1.49
Id:29360 %% El producto del supermercado MERCADONA :COLA NORMAL, COCA-COLA, BOTELLA 500 ml %% tiene un precio de €:1.45
Id:27326 %% El producto del superme

...

Id:46766 %% El producto del supermercado MERCADONA :CREMA CORPORAL ACEITE ARGAN, DELIPLUS, TARRO 250 ml %% tiene un precio de €:1.5
Id:35502 %% El producto del supermercado MERCADONA :CREMA CORPORAL ACEITE OLIVA, DELIPLUS, TARRO 250 ml %% tiene un precio de €:1.5
Id:60719 %% El producto del supermercado MERCADONA :CREMA CORPORAL AROMA VAINILLA, DELIPLUS, TARRO 250 g %% tiene un precio de €:1.8
Id:60842 %% El producto del supermercado MERCADONA :CREMA CORPORAL HIDRATANTE CON ACIDO HIALURONICO Y ESFERAS VITAMINA E (PARA TODO TIPO PIELES), DELIPLUS, TARRO 250 ml %% tiene un precio de €:2.5
Id:7907 %% El producto del supermercado MERCADONA :CREMA CORPORAL HIDRATANTE SORBETE MANGO (CONTIENE ACIDO HIALURONICO), DELIPLUS, TARRO 250 ml %% tiene un precio de €:2.5
Id:76542 %% El producto del supermercado MERCADONA :CREMA CORPORAL LATA AZUL, NIVEA, LATA 250 ml %% tiene un precio de €:4
Id:73984 %% El producto del supermercado MERCADONA :CREMA HOMBRE MEN PARA CARA, CUERPO Y MANOS, NIVEA, TARRO 15

Las subcategoria es PROTECCION E HIGIENE INTIMA
Voy a hacer click en PROTECCION E HIGIENE INTIMA
Las subsubcategorias son ['ASEO INTIMO', 'COMPRESAS', 'INCONTINENCIA', 'PAÑAL ADULTO Y EMPAPADORES', 'PROTEGESLIPS', 'TAMPONES', 'TOALLITAS']
Las primer subsubcategoria es ASEO INTIMO
Voy a hacer click en ASEO INTIMO
Id:44912 %% El producto del supermercado MERCADONA :GEL HIGIENE INTIMO LIQUIDO CON DOSIFICADOR, DELIPLUS, BOTELLA 500 ml %% tiene un precio de €:1.25
Id:44913 %% El producto del supermercado MERCADONA :GEL HIGIENE INTIMO LIQUIDO HIDRATANTE CON DOSIFICADOR, DELIPLUS, BOTELLA 500 ml %% tiene un precio de €:2.25

No se encontró información sobre el número de páginas.

Las primer subsubcategoria es COMPRESAS
Voy a hacer click en COMPRESAS
Id:47312 %% El producto del supermercado MERCADONA :COMPRESA ABSORCION NOCHE SUPER ULTRA PLEGADA ALAS, AUSONIA, PAQUETE 16 u %% tiene un precio de €:2.95
Id:86352 %% El producto del supermercado MERCADONA :COMPRESA ABSORCION NORMAL PLEGADA CON ALA

Id:79712 %% El producto del supermercado MERCADONA :TOALLITAS HIDROALCOHOLICAS (75 % ALCOHOL), DELIPLUS, PAQUETE 15 u %% tiene un precio de €:0.9
Id:47828 %% El producto del supermercado MERCADONA :TOALLITAS HUMEDAS DERMO HIGIENE CORPORAL (TOALLITAS GIGANTES), DELIPLUS, PAQUETE 60 u %% tiene un precio de €:3.95
Id:73989 %% El producto del supermercado MERCADONA :TOALLITAS HUMEDAS INTIMAS (SOBRE MONODOSIS), DELIPLUS, PAQUETE 20 u %% tiene un precio de €:1.8
Id:78975 %% El producto del supermercado MERCADONA :TOALLITAS HUMEDAS INTIMAS VIAJE, DELIPLUS, PAQUETE 12 u %% tiene un precio de €:0.85
Id:73988 %% El producto del supermercado MERCADONA :TOALLITAS HUMEDAS INTIMAS, DELIPLUS, PAQUETE 25 u %% tiene un precio de €:1.3
Id:76472 %% El producto del supermercado MERCADONA :TOALLITAS HUMEDAS REFRESCANTES TEJIDO SUAVE (SOBRE MONODOSIS), DELIPLUS, PAQUETE 20 u %% tiene un precio de €:1.75
Id:76457 %% El producto del supermercado MERCADONA :TOALLITAS HUMEDAS REFRESCANTES TEJIDO SUAVE, DELIPLUS

Id:24417 %% El producto del supermercado MERCADONA :CALAMAR INDIO CONGELADO GRANDE ENTERO, HACENDADO, u 200 APROX(PESO APROXIMADO DE LA UNIDAD 200 GR) %% tiene un precio de €:3.19
Id:87191 %% El producto del supermercado MERCADONA :CALAMAR REFRIGERADO PATAGONICO PEQUEÑO, HACENDADO, BANDEJA 300 g %% tiene un precio de €:5.1
Id:87189 %% El producto del supermercado MERCADONA :SEPIA REFRIGERADA LIMPIA TROCEADA, HACENDADO, BANDEJA 300 g %% tiene un precio de €:5.5
Id:87188 %% El producto del supermercado MERCADONA :SEPIA REFRIGERADA LIMPIA, HACENDADO, BANDEJA 375 g APROX(PESO APROXIMADO DE LA UNIDAD 375 GR) %% tiene un precio de €:5.60625
Id:87706 %% El producto del supermercado MERCADONA :SEPIA SUCIA REFRIGERADA, HACENDADO, GRANEL(PESO APROXIMADO DE LA UNIDAD 450 GR) %% tiene un precio de €:5.3775

No se encontró información sobre el número de páginas.

Las primer subsubcategoria es OTROS PESCADOS
Voy a hacer click en OTROS PESCADOS

No se encontró información sobre el número de páginas.


Las subcategoria es ZUMO REFRIGERADO
Voy a hacer click en ZUMO REFRIGERADO
No existen subsubcategorias, te guardo la subcategoria ZUMO REFRIGERADO
Id:39980 %% El producto del supermercado MERCADONA :SMOOTHIE FRESA REFRIGERADO, HACENDADO, BOTELLA 250 ml %% tiene un precio de €:1.5
Id:70636 %% El producto del supermercado MERCADONA :SMOOTHIE MANGO Y NARANJA REFRIGERADO, , BOTELLA 250 ml %% tiene un precio de €:1.5
Id:52632 %% El producto del supermercado MERCADONA :ZUMO 100 % NARANJA REFRIGERADO *****NOVEDAD*****, HACENDADO, BOTELLA PET 1 l %% tiene un precio de €:1.9
Id:86327 %% El producto del supermercado MERCADONA :ZUMO 100 % NARANJA REFRIGERADO *****NOVEDAD*****, HACENDADO, BOTELLA PET 330 ml %% tiene un precio de €:1
Id:39924 %% El producto del supermercado MERCADONA :ZUMO MANDARINA CLEMENTINA SELECCION REFRIGERADO, HACENDADO, BRICK 1 l %% tiene un precio de €:1.7
Id:39922 %% El producto del supermercado MERCADONA :ZUMO MANZANA SELECCION REFRIGERADO, HACENDADO, BRICK 1 l %% tiene u

In [38]:
with open(nombre_archivo_txt, "r") as archivo:
    contenido = archivo.read()  # Leer el contenido completo del archivo

# Dividir el contenido en fragmentos utilizando el separador //
fragmentos_texto = contenido.split("//")

# Crear documentos para cada fragmento con la metadata deseada
fragmentos = [Document(page_content=fragmento.strip(), metadata={"Supermercado": "Mercadona"}) for fragmento in fragmentos_texto if fragmento.strip()]



In [ ]:
def borrar_indices(index_name='todos'):
    import pinecone
    pc = Pinecone(api_key=PINECONE_API_KEY)

    if index_name == 'todos':
        indexes = pinecone.list_indexes()
        print('Borrando todos los índices ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Listo!')
    else:
        print(f'Borrando el índice: {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Listo')

### Creando Vectores (Embeddings) y subirlos a (Pinecone)

def creando_vectores(index_name,fragmentos):

    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model='text-embedding-ada-002')

    pc = Pinecone(api_key=PINECONE_API_KEY)

    if index_name in pc.list_indexes().names():
        index = pc.Index(index_name)
        print(f'El índice {index_name} ya existe. Cargando los embeddings ... ', end='')
        vectores = PineconeVectorStore.from_existing_index(index_name, embeddings)

        
    elif index_name not in pc.list_indexes().names():
        print(f'Creando el índice {index_name} y los embeddings ...', end='')
        pc.create_index(name=index_name,dimension=1536,metric="cosine", spec=ServerlessSpec(cloud="aws",region="us-east-1"))
        vectores = PineconeVectorStore.from_documents(fragmentos, embeddings, index_name=index_name)
        
        
    return vectores



In [ ]:
index_name = "mercadonafinal"
try:
    borrar_indices(index_name)
finally:
    creando_vectores(index_name,fragmentos)